In [ ]:
!pip install jieba

You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314482 sha256=39cf2cdaebcd128b1d6c9a9ea4752a12fdc2f6be13d13a7a7927e9b2e361c1fe
  Stored in directory: c:\users\lewis_yang\appdata\local\pip\cache\wheels\24\aa\17\5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba


In [ ]:
import jieba
list(jieba.cut(sentence = '這是第一篇文章.'))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.770 seconds.
DEBUG:jieba:Loading model cost 1.770 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['這是', '第一篇', '文章', '.']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
corpus = [
    '這是第一篇文章',
    '這文章是第二篇',
    '並且這是第三篇',
    '這是第一篇文章嗎',
]
words = []
for c in corpus :
    words.extend( list(jieba.cut(sentence = c )) )
np.unique( words )

array(['並且', '嗎', '文章', '是', '第一篇', '第三篇', '第二篇', '這', '這是'], dtype='<U3')

In [ ]:
# 手動轉換成 
# Sentence to vector 
#  [[0 1 1 1 0 0 1 0 1]
#  [0 2 0 1 0 1 1 0 1]
#  [1 0 0 1 1 0 1 1 1]
#  [0 1 1 1 0 0 1 0 1]]

list(jieba.cut(sentence = '這是第一篇文章' ))

['這是', '第一篇', '文章']

In [ ]:
# 思考
# ['這是', '第一篇', '文章'] 
# ['並且', '嗎', '文章', '是', '第一篇', '第三篇', '第二篇', '這', '這是']
# [0    ,    0,      1,    0,        1,       0,        0,   0,      1]
zh_all = np.unique( words )
zh_cv = np.zeros([4,9]).astype('int')
###############################################
#字頻分析表
##############################################
# 從文章中取出 每一 kk=序號， vv=每一條文章內容
for kk,vv in enumerate(corpus):
    # 針對 vv 每一個文章內容去切字
    for v in list(jieba.cut(sentence = vv )):
        # 針對 每一個單字 進行 轉換 1 
        pos = np.where(zh_all == v)
        zh_cv[kk, pos] = 1
zh_cv 

array([[0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 1, 0, 0, 0, 1]])

In [ ]:
zh_cv

array([[0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 1, 1, 0, 1, 0, 0, 0, 1]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(zh_cv[0].reshape(1,-1), zh_cv).ravel()

array([1.        , 0.28867513, 0.33333333, 0.8660254 ])

In [ ]:
# corpus = [
#     '這是第一篇文章',
#     '這文章是第二篇',
#     '並且這是第三篇',
#     '這是第一篇文章嗎',
# ]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
corpus = [
    '這是第一篇文章',
    '這文章是第二篇',
    '並且這是第三篇',
    '這是第一篇文章嗎',
]
words = []
for c in corpus :
    words.append( ' '.join(list(jieba.cut(sentence = c ))) )

print(words)

['這是 第一篇 文章', '這 文章 是 第二篇', '並且 這是 第三篇', '這是 第一篇 文章 嗎']


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(zh_cv)
tfidf.toarray().shape # 四篇文章 所有的單字 
tfidf.toarray() # 該部分類似 CountVectorizer() 顯示單字出現的頻率，加上權重

array([[0.        , 0.        , 0.53256952, 0.        , 0.65782931,
        0.        , 0.        , 0.        , 0.53256952],
       [0.        , 0.        , 0.34578314, 0.5417361 , 0.        ,
        0.        , 0.5417361 , 0.5417361 , 0.        ],
       [0.64450299, 0.        , 0.        , 0.        , 0.        ,
        0.64450299, 0.        , 0.        , 0.41137791],
       [0.        , 0.64065543, 0.40892206, 0.        , 0.5051001 ,
        0.        , 0.        , 0.        , 0.40892206]])

In [ ]:
base = 3
similarity_subject = np.array(cosine_similarity(tfidf[base], tfidf)).ravel()
similarity_subject

array([0.76782851, 0.14139835, 0.1682215 , 1.        ])